### Leer el dataframe guardado

In [2]:
import pandas as pd

def csv_to_dataframe(file_name):

    return pd.read_csv(file_name)

In [3]:
dataframe = csv_to_dataframe('./files/warped_terrain_sdf.csv')

In [4]:
dataframe

,x,y,z,density,distance
0,0.00,0.00,0.00,0.000000,0.070711
1,0.00,0.00,0.05,0.032586,0.050000
2,0.00,0.00,0.10,0.061833,-0.050000
3,0.00,0.00,0.15,0.082446,-0.100000
4,0.00,0.00,0.20,0.097867,-0.150000
...,...,...,...,...,...
7995,0.95,0.95,0.75,0.377797,0.860233
7996,0.95,0.95,0.80,0.366430,0.890225
7997,0.95,0.95,0.85,0.352210,0.921954
7998,0.95,0.95,0.90,0.334525,0.955249


### 5. Funcion para crear esferas

In [ ]:
len(dataframe)

In [ ]:
dataframe

In [ ]:
from math import sqrt

def distanceSquared3D(point, other):
    return sqrt((point[0]-other[0])**2 + (point[1]-other[1])**2 + (point[2]-other[2])**2)

def sphereIsInSphere(spheres, point, radio):
    for sphere in spheres: #sphere = [point, radio]
        distance = distanceSquared3D(point, sphere[0])
        if distance < sphere[1]:
            return True, 0
        elif distance < sphere[1]+radio:
            return True, distance-sphere[1]
    return False, radio

# take second element for sort
def takeSecond(elem):
    return elem[1]

def makeSpheres(dataframe, max_spheres): #dataframe = (x,y,z,density,distance)
    coordinates_spheres = [] #[(Point,r)]
    spheres_back = []
    
    for i in range(len(dataframe)):
        #print(i)
        distance = dataframe.iloc[i].distance
        if (distance < 0 and len(coordinates_spheres) < max_spheres):
            #print()
            coord = [dataframe.iloc[i].y, dataframe.iloc[i].x, dataframe.iloc[i].z]
            isInSphere, posibleRadio = sphereIsInSphere(coordinates_spheres, coord, -distance)
            if(not isInSphere):
                coordinates_spheres.append([coord, -distance])
            elif posibleRadio > 0:
                spheres_back.append([coord, posibleRadio])
                
    if len(coordinates_spheres) < max_spheres:
        spheres_back.sort(key=takeSecond, reverse=True)
        
    while len(coordinates_spheres) < max_spheres and i < len(spheres_back):
        coordinates_spheres.append([spheres_back[i]])
        i += 1
        
    return coordinates_spheres

def generateSpheres(dataframe, max_spheres, do_loop, use_brightness_level):
    sort_dataframe = dataframe.sort_values(["distance"])#, ascending=False)
    spheresPoints = makeSpheres(sort_dataframe, max_spheres)
    # plot(circles, use_brightness_level, do_loop)
    return spheresPoints

In [ ]:
spheres = generateSpheres(dataframe, 50, do_loop=False, use_brightness_level=True)

In [ ]:
spheres

### 5. Funcion para crear cubos

In [5]:
coordinates_cubes = [] #[(Point,L)]

In [6]:
def vertexInCube(vertex, X_min, X_max, Y_min, Y_max, Z_min, Z_max):
    X = vertex[0]
    Y = vertex[1]
    Z = vertex[2]
    
    return X_min <= X <= X_max and Y_min <= Y <= Y_max  and Z_min <= Z <= Z_max
    
    
def getXYZMax(point, side):
    X = point[0]
    Y = point[1]
    Z = point[2]

    X_max = X + side
    X_min = X - side

    Y_max = Y + side
    Y_min = Y - side

    Z_max = Z + side
    Z_min = Y - side
    
    return X_min, X_max, Y_min, Y_max, Z_min, Z_max
    
    
def cubeInCube(cube1, cube2): # cube = [(point), side]
    '''Return if part of cube1 is inside cube2'''
    point = cube2[0]
    side = cube2[1]
    X_min, X_max, Y_min, Y_max, Z_min, Z_max = getXYZMax(point, side)
    
    
    #######################################################

    point = cube1[0]
    side = cube1[1]
    X1_min, X1_max, Y1_min, Y1_max, Z1_min, Z1_max = getXYZMax(point, side)
    
    vertex1 = (X1_max, Y1_min, Z1_max)
    vertex2 = (X1_max, Y1_min, Z1_min)
    vertex3 = (X1_max, Y1_max, Z1_max)
    vertex4 = (X1_max, Y1_max, Z1_min)
    vertex5 = (X1_min, Y1_max, Z1_max)
    vertex6 = (X1_min, Y1_max, Z1_min)
    vertex7 = (X1_min, Y1_min, Z1_max)
    vertex8 = (X1_min, Y1_min, Z1_min)
    
    vertexes = [vertex1, vertex2, vertex3, vertex4, vertex5, vertex6, vertex7, vertex8]
    
    for vertex in vertexes:
        if(vertexInCube(vertex, X_min, X_max, Y_min, Y_max, Z_min, Z_max)):
            return True
    return False


def cubeIsInsideCube(cubes, point, side):
    originalCube = [point, side]
    for cube in cubes:
        if(cubeInCube(originalCube, cube) or cubeInCube(cube, originalCube)):
            return True
    return False



In [9]:
def makeCubes(dataframe, max_cubes): #dataframe = (x,y,z,density,distance)
    coordinates_cubes = [] #[(Point,r)]
    cubes_back = []
    
    for i in range(len(dataframe)):
        #print(i)
        distance = dataframe.iloc[i].distance
        if (distance < 0 and len(coordinates_cubes) < max_cubes):
            #print()
            coord = [dataframe.iloc[i].y, dataframe.iloc[i].x, dataframe.iloc[i].z]
            isInCube = cubeIsInsideCube(coordinates_cubes, coord, -distance)
            
            # isInSphere, posibleRadio = sphereIsInSphere(coordinates_spheres, coord, -distance)
            if(not isInCube):
                coordinates_cubes.append([coord, -distance])
#             elif posibleRadio > 0:
#                 spheres_back.append([coord, posibleRadio])
                
#     if len(coordinates_cubes) < max_cubes:
#         cubes_back.sort(key=takeSecond, reverse=True)
        
#     while len(coordinates_cubes) < max_cubes and i < len(cubes_back):
#         coordinates_cubes.append([cubes_back[i]])
#         i += 1
        
    return coordinates_cubes

def generateCubes(dataframe, max_cubes):
    sort_dataframe = dataframe.sort_values(["distance"])#, ascending=False)
    cubesPoints = makeCubes(sort_dataframe, max_cubes)
    return cubesPoints

In [10]:
cubes = generateCubes(dataframe, 50)

In [11]:
cubes

[[[0.95, 0.15000000000000002, 0.0], 0.1870828693386971],
 [[0.6000000000000001, 0.05, 0.15000000000000002], 0.15000000000000008],
 [[0.0, 0.0, 0.2], 0.15000000000000002],
 [[0.95, 0.1, 0.2], 0.15],
 [[0.95, 0.0, 0.25], 0.14142135623730948],
 [[0.6000000000000001, 0.0, 0.2], 0.12247448713915896],
 [[0.30000000000000004, 0.0, 0.2], 0.11180339887498948],
 [[0.1, 0.25, 0.0], 0.0866025403784439],
 [[0.25, 0.2, 0.0], 0.08660254037844385],
 [[0.05, 0.25, 0.05], 0.07071067811865478],
 [[0.0, 0.25, 0.05], 0.07071067811865478],
 [[0.35000000000000003, 0.25, 0.0], 0.07071067811865478],
 [[0.1, 0.25, 0.05], 0.07071067811865478],
 [[0.30000000000000004, 0.25, 0.0], 0.07071067811865478],
 [[0.05, 0.30000000000000004, 0.0], 0.07071067811865475],
 [[0.25, 0.25, 0.0], 0.07071067811865475],
 [[0.35000000000000003, 0.2, 0.05], 0.07071067811865475],
 [[0.15000000000000002, 0.25, 0.0], 0.07071067811865475],
 [[0.30000000000000004, 0.2, 0.05], 0.07071067811865475],
 [[0.25, 0.2, 0.05], 0.07071067811865474],

In [12]:
len(cubes)

50

### 6. Funcion para generar shader

In [ ]:
def getValueOfIndex(array, index):
    return array[index]

def getX(array):
    return getValueOfIndex(array, 0)

def getY(array):
    return getValueOfIndex(array, 1)

def getZ(array):
    return getValueOfIndex(array, 2)

Siendo
c = centro del eje
p = punto donde esta el cubo
s = tama;o del lado

##### Sacado de https://www.alanzucconi.com/2016/07/01/signed-distance-functions/
```
float sdf_box (float3 p, float3 c, float3 s){
    float x = max
    (   p.x - c.x - float3(s.x / 2., 0, 0),
        c.x - p.x - float3(s.x / 2., 0, 0)
    );
    
    float y = max
    (   p.y - c.y - float3(s.y / 2., 0, 0),
        c.y - p.y - float3(s.y / 2., 0, 0)
    );
    
    float z = max
    (   p.z - c.z - float3(s.z / 2., 0, 0),
        c.z - p.z - float3(s.z / 2., 0, 0)
    );
    
    float d = x;
    d = max(d,y);
    d = max(d,z);
    return d;
}
```

##### Esto es lo mismo que
```
float vmax(float3 v){
    return max(max(v.x, v.y), v.z);
}

float sdf_boxcheap(float3 p, float3 c, float3 s){
    return vmax(abs(p-c) - s);
}
```

In [ ]:
# ESTO NO ME ACUERDO DE DONDE LO SAQUE
/**
 * Signed distance function for a cube centered at the origin
 * with width = height = length = 2.0
 */
    pos - p
float cubeSDF(vec3 p) {
    // If d.x < 0, then -1 < p.x < 1, and same logic applies to p.y, p.z
    // So if all components of d are negative, then p is inside the unit cube
    vec3 d = abs(p) - vec3(1.0, 1.0, 1.0);
    
    // Assuming p is inside the cube, how far is it from the surface?
    // Result will be negative or zero.
    float insideDistance = min(max(d.x, max(d.y, d.z)), 0.0);
    
    // Assuming p is outside the cube, how far is it from the surface?
    // Result will be positive or zero.
    float outsideDistance = length(max(d, 0.0));
    
    return insideDistance + outsideDistance;
}

In [15]:
sum([1,2,3,5])

11

In [1]:
cubes = [[[0.95, 0.15000000000000002, 0.0], 0.1870828693386971],
 [[0.6000000000000001, 0.05, 0.15000000000000002], 0.15000000000000008],
 [[0.0, 0.0, 0.2], 0.15000000000000002],
 [[0.95, 0.1, 0.2], 0.15],
 [[0.95, 0.0, 0.25], 0.14142135623730948],
 [[0.6000000000000001, 0.0, 0.2], 0.12247448713915896],
 [[0.30000000000000004, 0.0, 0.2], 0.11180339887498948],
 [[0.1, 0.25, 0.0], 0.0866025403784439],
 [[0.25, 0.2, 0.0], 0.08660254037844385],
 [[0.05, 0.25, 0.05], 0.07071067811865478],
 [[0.0, 0.25, 0.05], 0.07071067811865478],
 [[0.35000000000000003, 0.25, 0.0], 0.07071067811865478],
 [[0.1, 0.25, 0.05], 0.07071067811865478],
 [[0.30000000000000004, 0.25, 0.0], 0.07071067811865478],
 [[0.05, 0.30000000000000004, 0.0], 0.07071067811865475],
 [[0.25, 0.25, 0.0], 0.07071067811865475],
 [[0.35000000000000003, 0.2, 0.05], 0.07071067811865475],
 [[0.15000000000000002, 0.25, 0.0], 0.07071067811865475],
 [[0.30000000000000004, 0.2, 0.05], 0.07071067811865475],
 [[0.25, 0.2, 0.05], 0.07071067811865474],
 [[0.1, 0.25, 0.1], 0.05000000000000002],
 [[0.0, 0.25, 0.1], 0.05000000000000002],
 [[0.05, 0.25, 0.1], 0.05000000000000002],
 [[0.30000000000000004, 0.25, 0.05], 0.05],
 [[0.65, 0.30000000000000004, 0.0], 0.05],
 [[0.35000000000000003, 0.25, 0.05], 0.05],
 [[0.2, 0.25, 0.0], 0.05],
 [[0.35000000000000003, 0.2, 0.1], 0.04999999999999999],
 [[0.25, 0.2, 0.1], 0.04999999999999999],
 [[0.30000000000000004, 0.2, 0.1], 0.04999999999999999],
 [[0.0, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.7000000000000001, 0.30000000000000004, 0.05], 0.04999999999999999],
 [[0.6000000000000001, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.55, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.5, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.45, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.15000000000000002, 0.25, 0.05], 0.04999999999999999],
 [[0.05, 0.35000000000000003, 0.0], 0.04999999999999999],
 [[0.4, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.35000000000000003, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.1, 0.30000000000000004, 0.05], 0.04999999999999999],
 [[0.1, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.05, 0.30000000000000004, 0.05], 0.04999999999999999],
 [[0.0, 0.30000000000000004, 0.05], 0.04999999999999999],
 [[0.15000000000000002, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.30000000000000004, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.25, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.7000000000000001, 0.35000000000000003, 0.0], 0.04999999999999999],
 [[0.2, 0.30000000000000004, 0.0], 0.04999999999999999],
 [[0.25, 0.25, 0.05], 0.04999999999999999]]

In [13]:
def generateShader(cubes, name = "cubeSDFFinal"):
#     f = """float cubeSDF(vec3 p) {
#     vec3 d = abs(p) - vec3(1.0, 1.0, 1.0);
#     return min(max(d.x,max(d.y,d.z)),0.0) + length(max(d,0.0));
# }"""
    f = """float vmax(vec3 v){
    \n     return max(max(v.x, v.y), v.z);
    \n}
    \n
    \nfloat sdf_boxcheap(vec3 c, vec3 p, vec3 s){
    \n     return vmax(abs(p-c) - s);
    \n}
    \n
    \nfloat """+name+"(vec3 inputPoint) {\nreturn "

    cube = cubes[0]
    x_max=1
    y_max=1
    z_max=1
    r_max=1
    prev = "sdf_boxcheap(inputPoint, vec3({0}, {1}, {2}), vec3({3}, {3}, {3}))".format(cube[0][0]/x_max, cube[0][1]/y_max, cube[0][2]/z_max, float(cube[1]/r_max))
    if (len(cubes) > 1):
        for i in range(1,len(cubes)):
            cube = cubes[i]
            opt = "min(sdf_boxcheap(inputPoint, vec3({0}, {1}, {2}), vec3({3}, {3}, {3})), {4})".format(cube[0][0]/x_max, cube[0][1]/y_max, cube[0][2]/z_max, float(cube[1]/r_max), prev)
            prev = opt
        f += opt
    else:
        f += prev
    f += ";\n}"
    print(f)

In [16]:
def generateShader(cubes, name = "cubeSDFFinal"):
#     f = """float cubeSDF(vec3 p) {
#     vec3 d = abs(p) - vec3(1.0, 1.0, 1.0);
#     return min(max(d.x,max(d.y,d.z)),0.0) + length(max(d,0.0));
# }"""
    f = """float vmax(vec3 v){
    \n     return max(max(v.x, v.y), v.z);
    \n}
    \n
    \nfloat sdf_boxcheap(vec3 c, vec3 p, vec3 s){
    \n     return vmax(abs(p-c) - s);
    \n}
    \n
    \nfloat """+name+"(vec3 inputPoint) {\nreturn "

    cube = cubes[0]
    x_max=1
    y_max=1
    z_max=1
    r_max=1
    prev = "sdf_boxcheap(inputPoint, vec3({0}, {1}, {2}), vec3({3}, {3}, {3}))".format(cube[0][0]/x_max, cube[0][1]/y_max, cube[0][2]/z_max, float(cube[1]/r_max))
    if (len(cubes) > 1):
        for i in range(1,len(cubes)):
            cube = cubes[i]
            opt = "smoothUnion(sdf_boxcheap(inputPoint, vec3({0}, {1}, {2}), vec3({3}, {3}, {3})), {4})".format(cube[0][0]/x_max, cube[0][1]/y_max, cube[0][2]/z_max, float(cube[1]/r_max), prev)
            prev = opt
        f += opt
    else:
        f += prev
    f += ";\n}"
    print(f)

In [17]:
generateShader(cubes)

float vmax(vec3 v){
    
     return max(max(v.x, v.y), v.z);
    
}
    

    
float sdf_boxcheap(vec3 c, vec3 p, vec3 s){
    
     return vmax(abs(p-c) - s);
    
}
    

    
float cubeSDFFinal(vec3 inputPoint) {
return smoothUnion(sdf_boxcheap(inputPoint, vec3(0.25, 0.25, 0.05), vec3(0.04999999999999999, 0.04999999999999999, 0.04999999999999999)), smoothUnion(sdf_boxcheap(inputPoint, vec3(0.2, 0.30000000000000004, 0.0), vec3(0.04999999999999999, 0.04999999999999999, 0.04999999999999999)), smoothUnion(sdf_boxcheap(inputPoint, vec3(0.7000000000000001, 0.35000000000000003, 0.0), vec3(0.04999999999999999, 0.04999999999999999, 0.04999999999999999)), smoothUnion(sdf_boxcheap(inputPoint, vec3(0.25, 0.30000000000000004, 0.0), vec3(0.04999999999999999, 0.04999999999999999, 0.04999999999999999)), smoothUnion(sdf_boxcheap(inputPoint, vec3(0.30000000000000004, 0.30000000000000004, 0.0), vec3(0.04999999999999999, 0.04999999999999999, 0.04999999999999999)), smoothUnion(sdf_boxcheap(inputPoint, 

In [ ]:
# def generateShader(spheres, number):
#     name =  "sphereSDF{}".format(number)
#     f = "float vmax(float3 v){
#     return max(max(v.x, v.y), v.z);
# }

# float sdf_boxcheap(float3 p, float3 c, float3 s){
#     return vmax(abs(p-c) - s);
# }
#     function = "/**
#  * Signed distance function for a cube centered at the origin
#  * with width = height = length = 2.0
#  */
# "float cubeSDF(vec3 p, vec3 pos) {" +
# "\n    // If d.x < 0, then -1 < p.x < 1, and same logic applies to p.y, p.z" +
# "\n    // So if all components of d are negative, then p is inside the unit cube" +
# "\n    vec3 d = abs(pos - p) - vec3(1.0, 1.0, 1.0);" +
# "\n    // Assuming p is inside the cube, how far is it from the surface?" +
# "\n    // Result will be negative or zero." +
# "\n    float insideDistance = min(max(d.x, max(d.y, d.z)), 0.0);" +
    
# "\n    // Assuming p is outside the cube, how far is it from the surface?" +
# "\n    // Result will be positive or zero." +
# "\n    float outsideDistance = length(max(d, 0.0));" +
    
# "\n    return insideDistance + outsideDistance;" +
# "\n}" +
#     f = "float sphereSD(vec3 p, vec3 pos, float r) { return length(pos - p) - r; }\nfloat "+name+"(vec3 inputPoint) {\nreturn "

#     sphere = spheres[0]
#     x_max=1
#     y_max=1
#     z_max=1
#     r_max=1
#     prev = "sphereSD(inputPoint, vec3({0}, {1}, {2}), {3})".format(sphere[0][0]/x_max, sphere[0][1]/y_max, sphere[0][2]/z_max, float(sphere[1]/r_max))
#     if (len(spheres) > 1):
#         for i in range(1,len(spheres)):
#             sphere = spheres[i]
#             opt = "min(sphereSD(inputPoint, vec3({0}, {1}, {2}), {3}), {4})".format(sphere[0][0]/x_max, sphere[0][1]/y_max, sphere[0][2]/z_max, float(sphere[1]/r_max), prev)
#             prev = opt
#         f += opt
#     else:
#         f += prev
#     f += ";\n}"
#     print(f)